In [1]:
# ! pip install transformers

In [2]:
# ! pip install google-cloud-storage

In [4]:
# ! pip install torch
# ! pip install peft
# ! pip install tiktoken matplotlib pillow
# ! pip install einops transformers_stream_generator
# ! pip install torchvision

In [1]:
# from google.cloud import storage
# import os

# # Initialize a client using your service account key
# client = storage.Client.from_service_account_json('/home/ubuntu/keen-snow-373818-93f44c794bab.json')

# # Define your bucket name and folder to download
# bucket_name = 'carla_dataset_bucket'
# folder_prefix = 'output_carla_25K_automated/'  # Ensure the folder name ends with a '/'
# local_download_path = '/home/ubuntu/output_carla_25K_automated'

# # Folders to ignore
# ignore_folders = ['checkpoint-1000/', 'checkpoint-2000/', 'checkpoint-3000/']

# bucket = client.get_bucket(bucket_name)

# # List blobs (files) in the folder
# blobs = bucket.list_blobs(prefix=folder_prefix)

# # Download each file in the folder
# for blob in blobs:
#     # Skip blobs in the ignore folders
#     if any(blob.name.startswith(folder_prefix + ignore_folder) for ignore_folder in ignore_folders):
#         continue

#     # Construct the local file path
#     local_file_path = os.path.join(local_download_path, blob.name[len(folder_prefix):])

#     # Create directories if they don't exist
#     os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

#     # Download the file
#     blob.download_to_filename(local_file_path)
#     print(f"Downloaded {blob.name} to {local_file_path}")

# print("Download completed.")

Downloaded output_carla_25K_automated/README.md to /home/ubuntu/output_carla_25K_automated/README.md
Downloaded output_carla_25K_automated/adapter_config.json to /home/ubuntu/output_carla_25K_automated/adapter_config.json
Downloaded output_carla_25K_automated/adapter_model.bin to /home/ubuntu/output_carla_25K_automated/adapter_model.bin
Downloaded output_carla_25K_automated/qwen.tiktoken to /home/ubuntu/output_carla_25K_automated/qwen.tiktoken
Downloaded output_carla_25K_automated/special_tokens_map.json to /home/ubuntu/output_carla_25K_automated/special_tokens_map.json
Downloaded output_carla_25K_automated/tokenizer_config.json to /home/ubuntu/output_carla_25K_automated/tokenizer_config.json
Downloaded output_carla_25K_automated/trainer_state.json to /home/ubuntu/output_carla_25K_automated/trainer_state.json
Downloaded output_carla_25K_automated/training_args.bin to /home/ubuntu/output_carla_25K_automated/training_args.bin
Download completed.


In [1]:
import sys

# This will print the full path to the Python interpreter executable
print(sys.executable)

/home/ubuntu/Carla_server/modelenv/bin/python


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch

torch.manual_seed(1234)

# # Path to the local directory where the model files are stored
local_model_path = '/home/ubuntu/output_carla_25K_automated'

from peft import AutoPeftModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat", trust_remote_code=True)

model = AutoPeftModelForCausalLM.from_pretrained(
    local_model_path, # path to the output directory
    device_map="auto",
    trust_remote_code=True
).eval()

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [3]:
# query = "Picture 1: <img>/home/ubuntu/Carla_server/Carla_Repo/TCP/data/results_LLVM/routes_lav_valid_12_11_16_00_21/input_image/0001.png</img>\\n Given this Ego centric image, now you are autonomous driving agent, now your current position(x,y) is at (122.055,-120.844) \\n Your current speed is 1.268 m/s \\n Your current angle is 6.279 \\n Your immediate command to follow is FOLLOW THE CURRENT LANE and immediate target position is at (127.398,-120.813) \\n Your later target command to follow is GO RIGHT and later target position is at (127.398,-120.813) \\n The current command for applying brakes is False \\n Now you have to predict following values which are throttle (also called acceleration) (0 to 1) , steer (-1 to 1) and brake (0 to 1)\\n in the json format For example: the output can be this json { \"throttle\" :x, \"steer\" : y, \"brake\": z }"
# response, history = model.chat(tokenizer, query=query, history=None)
# print(response)

In [4]:
import socket
import time
import json
import ast

def create_socket():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    return s

def listen_for_initiator(host, port):
    sock = create_socket()
    sock.bind((host, port))
    sock.listen(1)
    conn, addr = sock.accept()
    return conn

def connect_to_initiator(host, port):
    sock = create_socket()
    while True:
        try:
            sock.connect((host, port))
            return sock
        except ConnectionRefusedError:
            time.sleep(1)  # Wait a bit before trying to reconnect

In [5]:
host = '127.0.0.1'
port = 12345

mapping = {1:"GO LEFT" , 2: "GO RIGHT" ,  3: "GO STRAIGHT" , 4: "FOLLOW THE CURRENT LANE" , 5: "CHANGE TO LANE LEFT" , 6: "CHANGE TO LANE RIGHT" }

print("Responder listening for Initiator...")
responder_socket = listen_for_initiator(host, port)




while True:

    data = responder_socket.recv(1024)

    SEND_DATA = json.loads(data.decode())
    print(SEND_DATA)

    image_path = "/home/ubuntu/Carla_server/Carla_Repo/TCP/"+ SEND_DATA["rgb"]

    # query = f'Picture 1: <img>{image_path}</img> Given this Ego centric image, now you are autonomous driving agent, now your current position(x,y) is at ({SEND_DATA["x"]},{SEND_DATA["y"]}) Your current speed is {SEND_DATA["speed"]} m/s Your current angle is {SEND_DATA["theta"]} Your immediate command to follow is {mapping[SEND_DATA["command_near"]]} and immediate target position is at ({SEND_DATA["x_command_near"]},{SEND_DATA["y_command_near"]}) Your later target command to follow is {mapping[SEND_DATA["target_command"]]} and later target position is at ({SEND_DATA["x_target"]},{SEND_DATA["y_target"]}) The current command for applying brakes is {SEND_DATA["should_brake"]} Now you have to predict following values which are throttle (also called acceleration) (0 to 1) , steer (-1 to 1) and brake (0 to 1) in the json format For example: the output can be this json {{ "throttle" :x, "steer" : y, "brake": z }}'
    query = f'Picture 1: <img>{image_path}</img>\\n Given this Ego centric image, \
            now you are autonomous driving agent, now your current position(x,y) is at \
            ({round(SEND_DATA["x"],3)},{round(SEND_DATA["y"],3)}) \\n \
            Your current speed is {round(SEND_DATA["speed"],3)} m/s \\n \
            Your current angle is {round(SEND_DATA["theta"],3)}  \\n \
            Your immediate command to follow is {mapping[SEND_DATA["command_near"]]} and immediate target position is at \
            ({round(SEND_DATA["x_command_near"],3)},{round(SEND_DATA["y_command_near"],3)}) \\n \
            Your later target command to follow is {mapping[SEND_DATA["target_command"]]} and later target position is at \
            ({round(SEND_DATA["x_target"],3)},{round(SEND_DATA["y_target"],3)}) \\n \
            The current command for applying brakes is {SEND_DATA["should_brake"]} \\n \
            Now you have to predict following values which are throttle (also called acceleration) (0 to 1) , steer (-1 to 1) and brake (0 to 1)\\n in the json format \
            For example: the output can be this json {{ "throttle" :x, "steer" : y, "brake": z }}'
    print(query)
    response, history = model.chat(tokenizer, query=query, history=None)
    print(response)

    response = ast.literal_eval(response)
    response = json.dumps(response)

    responder_socket.sendall(response.encode())

Responder listening for Initiator...
{'rgb': 'data/results_LLVM/routes_lav_valid_12_18_02_14_54/input_image/0003.png', 'x': -0.001610136389302852, 'y': -3.104682434048109e-05, 'theta': 6.282840251922607, 'speed': 0.0, 'x_command_near': -175.24763159398083, 'y_command_near': -3.4570198106200576, 'command_near': 4, 'should_brake': True, 'x_target': -127.25046896413592, 'y_target': -3.381767515977085, 'target_command': 4}
Picture 1: <img>/home/ubuntu/Carla_server/Carla_Repo/TCP/data/results_LLVM/routes_lav_valid_12_18_02_14_54/input_image/0003.png</img>\n Given this Ego centric image,             now you are autonomous driving agent, now your current position(x,y) is at             (-0.002,-0.0) \n             Your current speed is 0.0 m/s \n             Your current angle is 6.283  \n             Your immediate command to follow is FOLLOW THE CURRENT LANE and immediate target position is at             (-175.248,-3.457) \n             Your later target command to follow is FOLLOW THE CUR

ConnectionResetError: [Errno 104] Connection reset by peer

In [ ]:
name = "{ 'throttle' :0.0, 'steer' : -0.005966780083557129, 'brake': 1.0 }"
import ast 
print(type(ast.literal_eval(name)))